### Reflection
Allows a program to inspect itself during runtime. `java.lang.Class` provides methods to examine the runtime properties of the object including its members and type information. Class also provides the ability to create new classes and objects. Most importantly, it is the entry point for all of the Reflection APIs.  

Two common syntaxes are used to refer to the Class object - `.class` and `getClass`. Both are quite different, so we should keep in mind the difference. The first one is static reference and resolves at compile time, whereas the other resolves at runtime.

In [2]:
Class strClass = "Hello".getClass();

// Main usage of the below syntax are cases where we don't have an
// instance, we just have the Class
Class strClass2 = String.class;

System.out.println(strClass);
System.out.println(strClass2);

class java.lang.String
class java.lang.String


In [3]:
CharSequence characters = new String("Hello");
Class clazz = characters.getClass();

System.out.println(clazz);

class java.lang.String


Another way to get the Class is `Class.forName`. But again, this is used when we have the fully qualified name of the class, not when we have an instance of the class.

In [4]:
Class clazz = Class.forName("java.lang.String");

Class clazz = characters.getClass();

System.out.println(clazz);

class java.lang.String


### Class Modifiers and Types
To get modifiers `public`, `private`, `protected`, `static`, `final`, `abstract`, etc and to get annotation data, and to get other class information:

In [4]:
import java.lang.reflect.*;
import java.util.*;

Class<?> clazz = "String".getClass();

// Get class name
System.out.println("Class: " + clazz.getCanonicalName());

// Modifiers
System.out.println("Modifiers: " + Modifier.toString(clazz.getModifiers()));

// Ancestors
System.out.println("Ancestors: ");
Class<?> ancestor = clazz.getSuperclass();
System.out.println("\t" + ancestor.getCanonicalName());
while(ancestor != Object.class){
    ancestor = clazz.getSuperclass();
    System.out.println("\t" + ancestor.getCanonicalName());
}

Class: java.lang.String
Modifiers: public final
Ancestors: 
	java.lang.Object


### Class Members
We can get constructors, instance variables and methods of a class using the Reflectoin API. `Class` methods having *Declared* in its name means exclude inherited members.

In [5]:
Class<?> clazz = "Integer".getClass();

// All declared instance variables:
System.out.println("Fields:");
Field[] fields = clazz.getDeclaredFields();
for (Field f : fields) {
    System.out.print("\t" + f.getName());
}

// All declared instance methods:
System.out.println("\nMethods:");
Method[] methods = clazz.getDeclaredMethods();
for (Method m : methods) {
    System.out.print("\t" + m.getName());
}

// All declared instance methods:
Constructor<?>[] constructors = clazz.getDeclaredConstructors();
System.out.println("\nConstructors: " + constructors.length + " count");

Fields:
	value	coder	hash	hashIsZero	serialVersionUID	COMPACT_STRINGS	serialPersistentFields	CASE_INSENSITIVE_ORDER	LATIN1	UTF16
Methods:
	value	equals	length	toString	hashCode	getChars	compareTo	compareTo	indexOf	indexOf	indexOf	indexOf	indexOf	checkIndex	valueOf	valueOf	valueOf	valueOf	valueOf	valueOf	valueOf	valueOf	valueOf	coder	rangeCheck	checkBoundsOffCount	isLatin1	charAt	codePointAt	codePointBefore	codePointCount	offsetByCodePoints	checkBoundsBeginEnd	getBytes	getBytes	getBytes	getBytes	getBytes	contentEquals	contentEquals	nonSyncContentEquals	regionMatches	regionMatches	startsWith	startsWith	lastIndexOf	lastIndexOf	lastIndexOf	lastIndexOf	lastIndexOf	substring	substring	isEmpty	replace	replace	matches	replaceFirst	replaceAll	split	split	toLowerCase	toLowerCase	toUpperCase	toUpperCase	trim	strip	stripLeading	stripTrailing	indexOfNonWhitespace	lines	repeat	lastIndexOfNonWhitespace	outdent	isBlank	toCharArray	format	format	resolveConstantDesc	resolveConstantDesc	codePoints	equals